**LIVEFACE DETECTION ON MOBILE DEVICES**

**AIM**

The goal of this task is to identify liveface by different types of mobile cameras

Face Detection Algorithms : 

1. OpenCV Haar Cascade
2. ResNet
3. FindFace

# Import Modules

In [1]:
import os
from os import listdir
from os.path import join, isfile

In [2]:
import math
from collections import Counter

In [3]:
import cv2
import numpy as np
import pandas as pd

from tqdm import tqdm

In [4]:
tqdm.pandas()

# Load Data

In [5]:
dataset = pd.read_csv('FINAL_ALL_LIVEFACE_DATASET_V2.csv')

In [6]:
dataset.head()

,image_path,image_metadata,label,set_status
0,LIVEFACE_DATA/ATM_DATASET/LIVE/KASPI_ATM_VIDEO...,LIVEFACE_DATA/ATM_DATASET/LIVE/KASPI_ATM_VIDEO...,LIVE,Train
1,LIVEFACE_DATA/ATM_DATASET/LIVE/KASPI_ATM_VIDEO...,LIVEFACE_DATA/ATM_DATASET/LIVE/KASPI_ATM_VIDEO...,LIVE,Train
2,LIVEFACE_DATA/ATM_DATASET/LIVE/KASPI_ATM_VIDEO...,LIVEFACE_DATA/ATM_DATASET/LIVE/KASPI_ATM_VIDEO...,LIVE,Train
3,LIVEFACE_DATA/ATM_DATASET/LIVE/KASPI_ATM_VIDEO...,LIVEFACE_DATA/ATM_DATASET/LIVE/KASPI_ATM_VIDEO...,LIVE,Train
4,LIVEFACE_DATA/ATM_DATASET/LIVE/KASPI_ATM_VIDEO...,LIVEFACE_DATA/ATM_DATASET/LIVE/KASPI_ATM_VIDEO...,LIVE,Train


In [7]:
dataset['image_path'][0]

'LIVEFACE_DATA/ATM_DATASET/LIVE/KASPI_ATM_VIDEO_STATE_6/full_20180227111503272000B6079.png'

# Face Detection and Cropping

## OpenCV HaarCascade

In [8]:
CASCADE_PATH = 'Model_Data/haarcascade_frontalface_default.xml'

In [9]:
face_cascade_clf = cv2.CascadeClassifier(CASCADE_PATH)

In [10]:
def calculate_distance(p1,p2):

    return math.sqrt((p1[0] - p2[0])**2 + (p1[1] - p2[1])**2) 

In [11]:
def haarcascade_faces(path_org,path_save):
    
    img = cv2.imread(path_org)
        
    result_shape = (0,0,0)
    
    faces = face_cascade_clf.detectMultiScale(img, 1.1, 3, minSize = (100,100))
    
    if (img is None):
        return result_shape
    
    if (faces is None):
        return result_shape
    
    elif len(faces) > 0:
        
        height,width = img.shape[:2]
        central_point = (height / 2, width / 2)
        
        central_dist = 100000
        central_face = None
        
        for (x, y, w, h) in faces:
            r = max(w, h) / 2
            centerx = x + w / 2
            centery = y + h / 2
            nx = int(centerx - r)
            ny = int(centery - r)
            nr = int(r * 2)
            
            faceimg = img[ny:ny+nr, nx:nx+nr]

            face_distance = calculate_distance(p1 = central_point, p2 = (nx,ny)) 
            
#             print(face_distance)

            if face_distance < central_dist:
                central_dist = face_distance
                central_face = cv2.resize(faceimg, (256, 256))
        
        cv2.imwrite(filename = path_save, img = central_face)
        
        
        return central_face.shape if (central_face is not None) else result_shape 
    

In [12]:
def create_path(path_data):
    for path in path_data:
        os.makedirs(path)
    return None

In [13]:
dataset = dataset.dropna()

In [14]:
len(dataset)

128061

In [24]:
dataset['openCV_haarcascade'] = dataset['image_path'].progress_apply(haarcascade_faces)

100%|██████████| 128061/128061 [48:32<00:00, 43.97it/s] 


In [25]:
dataset['openCV_haarcascade'].value_counts()

(256, 256, 3)    114088
(0, 0, 0)         13973
Name: openCV_haarcascade, dtype: int64

In [32]:
dataset[dataset['openCV_haarcascade'] == (0,0,0)]['image_path'][127552]

'LIVEFACE_DATA/MOB_DATASET_NORMAL/FAKE/PHONE/STATE_1/20180724_103659.jpg'

In [15]:
dataset['image_metadata_cropped'] = dataset['image_metadata'].apply(lambda x : x.replace('LIVEFACE_DATA','LIVEFACE_DATA_CROPPED'))

In [16]:
dataset['image_path_cropped'] = dataset['image_path'].apply(lambda x : x.replace('LIVEFACE_DATA','LIVEFACE_DATA_CROPPED'))

In [17]:
dataset['image_path_cropped'][0]

'LIVEFACE_DATA_CROPPED/ATM_DATASET/LIVE/KASPI_ATM_VIDEO_STATE_6/full_20180227111503272000B6079.png'

In [18]:
create_path(path_data = dataset['image_metadata_cropped'].unique())

In [53]:
dataset.to_csv('FINAL_ALL_LIVEFACE_DATASET_V3.csv', index = False)

In [19]:
shapes = []

for i in tqdm(range(len(dataset))):
    shape = haarcascade_faces(path_org = dataset['image_path'][i], path_save = dataset['image_path_cropped'][i])
    shapes.append(shape)

100%|██████████| 128061/128061 [51:03<00:00,  5.84it/s] 


In [20]:
Counter(shapes)

Counter({(256, 256, 3): 114088, None: 13968, (0, 0, 0): 5})

In [24]:
len(shapes)

128061

In [21]:
dataset['openCV_haarcascade'] = shapes

In [22]:
dataset.head()

,image_path,image_metadata,label,set_status,image_metadata_cropped,image_path_cropped,openCV_haarcascade
0,LIVEFACE_DATA/ATM_DATASET/LIVE/KASPI_ATM_VIDEO...,LIVEFACE_DATA/ATM_DATASET/LIVE/KASPI_ATM_VIDEO...,LIVE,Train,LIVEFACE_DATA_CROPPED/ATM_DATASET/LIVE/KASPI_A...,LIVEFACE_DATA_CROPPED/ATM_DATASET/LIVE/KASPI_A...,"(256, 256, 3)"
1,LIVEFACE_DATA/ATM_DATASET/LIVE/KASPI_ATM_VIDEO...,LIVEFACE_DATA/ATM_DATASET/LIVE/KASPI_ATM_VIDEO...,LIVE,Train,LIVEFACE_DATA_CROPPED/ATM_DATASET/LIVE/KASPI_A...,LIVEFACE_DATA_CROPPED/ATM_DATASET/LIVE/KASPI_A...,"(256, 256, 3)"
2,LIVEFACE_DATA/ATM_DATASET/LIVE/KASPI_ATM_VIDEO...,LIVEFACE_DATA/ATM_DATASET/LIVE/KASPI_ATM_VIDEO...,LIVE,Train,LIVEFACE_DATA_CROPPED/ATM_DATASET/LIVE/KASPI_A...,LIVEFACE_DATA_CROPPED/ATM_DATASET/LIVE/KASPI_A...,"(256, 256, 3)"
3,LIVEFACE_DATA/ATM_DATASET/LIVE/KASPI_ATM_VIDEO...,LIVEFACE_DATA/ATM_DATASET/LIVE/KASPI_ATM_VIDEO...,LIVE,Train,LIVEFACE_DATA_CROPPED/ATM_DATASET/LIVE/KASPI_A...,LIVEFACE_DATA_CROPPED/ATM_DATASET/LIVE/KASPI_A...,"(256, 256, 3)"
4,LIVEFACE_DATA/ATM_DATASET/LIVE/KASPI_ATM_VIDEO...,LIVEFACE_DATA/ATM_DATASET/LIVE/KASPI_ATM_VIDEO...,LIVE,Train,LIVEFACE_DATA_CROPPED/ATM_DATASET/LIVE/KASPI_A...,LIVEFACE_DATA_CROPPED/ATM_DATASET/LIVE/KASPI_A...,"(256, 256, 3)"


In [25]:
dataset.dropna(inplace = True)

In [26]:
dataset.shape

(114093, 7)

In [28]:
dataset['openCV_haarcascade'].value_counts()

(256, 256, 3)    114088
(0, 0, 0)             5
Name: openCV_haarcascade, dtype: int64

In [30]:
dataset = dataset[dataset['openCV_haarcascade'] == (256,256,3)].reset_index().drop('index', axis = 1)

In [32]:
dataset['set_status'].value_counts()

Train         108729
Validation      5359
Name: set_status, dtype: int64

In [33]:
dataset.to_csv('FINAL_ALL_LIVEFACE_DATASET_V4.csv')

## ResNet

# FindFace